# Import

In [1]:
import os

import numpy as np
import pandas as pd

from sklearn.metrics import r2_score

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# from torchvision.models import resnet50, ResNet50_Weights, resnet34, ResNet34_Weights, inception_v3, Inception_V3_Weights

from torchvision.models import vit_b_16, ViT_B_16_Weights

# Data

In [2]:
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

'''from PIL import Image

image1 = Image.open("/kaggle/input/cs-480-2024-spring/data/train_images/190966811.jpeg")
image2 = Image.open("/kaggle/input/cs-480-2024-spring/data/test_images/179127153.jpeg")

print(image1.mode)
print(image1.size)
print(image2.mode)
print(image2.size)'''

'from PIL import Image\n\nimage1 = Image.open("/kaggle/input/cs-480-2024-spring/data/train_images/190966811.jpeg")\nimage2 = Image.open("/kaggle/input/cs-480-2024-spring/data/test_images/179127153.jpeg")\n\nprint(image1.mode)\nprint(image1.size)\nprint(image2.mode)\nprint(image2.size)'

In [3]:
train_data = pd.read_csv("/kaggle/input/cs-480-2024-spring/data/train.csv")
train_data = train_data.set_index('id')
validation_data = train_data.sample(frac = 0.1)
train_data =  train_data.drop(validation_data.index)

train_traits = train_data.iloc[:,-6:]
train_data = train_data.iloc[:,:-6]

validation_traits = validation_data.iloc[:,-6:]
validation_data = validation_data.iloc[:,:-6]

In [4]:
'''print(len(train_data), len(validation_data))
print(train_traits.head())
print(validation_traits.head())'''

'print(len(train_data), len(validation_data))\nprint(train_traits.head())\nprint(validation_traits.head())'

## Data transformation

In [5]:
# Log 
train_traits_log = np.log10(train_traits)
validation_traits_log = np.log10(validation_traits)

# Values
traits_max = train_traits_log.max()
traits_min = train_traits_log.min()
traits_mean = train_traits.mean()
traits_std = train_traits.std()

data_max = train_data.max()
data_min = train_data.min()
data_mean = train_data.mean()
data_std = train_data.std()

# Normalize
train_traits_normalized = (train_traits_log - traits_min) / (traits_max - traits_min)
validation_traits_normalized = (validation_traits_log - traits_min) / (traits_max - traits_min)

train_data_normalized = (train_data - data_min) / (data_max - data_min)
validation_data_normalized = (validation_data - data_min) / (data_max - data_min)

def inverse_transformation(trait):
    return np.power(10, (trait * (traits_max - traits_min)) + traits_min)

In [6]:
'''print(train_traits_normalized.head())
print(validation_traits_normalized.head())
print(train_data_normalized.head())
print(validation_data_normalized.head())'''

'''
print(test_data.head())
print(test_traits.head())'''

'''print(train_traits.loc[101801795])
print(type(train_traits.loc[101801795]))'''

'''print(torch.tensor(train_traits.loc[101801795].values))'''

# print(data_max, data_min)

'print(torch.tensor(train_traits.loc[101801795].values))'

In [7]:
class CustomDataSet(Dataset):
    def __init__(self, root_dir, indices, label_mapping, target_mapping, transform = None):
        self.root_dir = root_dir
        self.indices = indices
        self.transform = transform
        self.image_paths = []
        self.targets = []
        self.labels = []
        
        for idx in indices:
            self.image_paths.append(os.path.join(root_dir, str(idx) + ".jpeg"))
            target = self.get_target_from_filename(idx, target_mapping)
            self.targets.append(target)
            label = self.get_label_from_filename(idx, label_mapping)
            self.labels.append(label)
                
    def get_target_from_filename(self, idx, target_mapping):
        target = target_mapping.loc[idx].values
        return torch.tensor(target).float()
    
    def get_label_from_filename(self, idx, label_mapping):
        target = label_mapping.loc[idx].values
        return torch.tensor(target).float()
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        target = self.targets[idx]
        label = self.labels[idx]
 
        if self.transform:
            image = self.transform(image)

        return image, label, target

In [8]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=(0.9, 1.1), contrast=(0.9, 1.1), saturation=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

validation_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


trainset = CustomDataSet('/kaggle/input/cs-480-2024-spring/data/train_images/', train_data_normalized.index.tolist(), train_data_normalized,
                          train_traits_normalized, transform=train_transform)
validationset = CustomDataSet('/kaggle/input/cs-480-2024-spring/data/train_images/', validation_data_normalized.index.tolist(), validation_data_normalized,
                               validation_traits_normalized, transform=validation_transform)

batch_size = 32

trainloader = DataLoader(trainset, batch_size = batch_size, shuffle=True)
validationloader = DataLoader(validationset, batch_size = batch_size, shuffle=False)

In [9]:
'''for i, sample in enumerate(trainloader):
    image, label, target = sample[0], sample[1], sample[2]
    print(image, label, target)
    break'''

# print(len(trainloader))

'for i, sample in enumerate(trainloader):\n    image, label, target = sample[0], sample[1], sample[2]\n    print(image, label, target)\n    break'

# CNN

In [10]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## Model

In [11]:
'''weights = ViT_B_16_Weights.DEFAULT
preprocess = weights.transforms()

print(preprocess)'''

# print(vit_b_16(weights=ViT_B_16_Weights.DEFAULT))

'weights = ViT_B_16_Weights.DEFAULT\npreprocess = weights.transforms()\n\nprint(preprocess)'

In [12]:
class CNNBranch(nn.Module):
    def __init__(self):
        super(CNNBranch, self).__init__()
        
        self.pretrained = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
        
        for param in self.pretrained.parameters():
            param.requires_grad = False
            
        for module in self.pretrained.modules():
            if isinstance(module, nn.LayerNorm):
                for param in module.parameters():
                    param.requires_grad = True
        
        fc_inputs = 1000
        self.image = nn.Sequential(
            nn.Linear(fc_inputs, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
    
    def forward(self, x):
        x = self.pretrained(x)
        x = self.image(x)
        return x
    

class AncillaryBranch(nn.Module):
    def __init__(self):
        super(AncillaryBranch, self).__init__()
        
        self.ancillary = nn.Sequential(
            nn.Linear(163, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
    
    def forward(self, x):
        x = self.ancillary(x)
        return x


class CombinedModel(nn.Module):
    def __init__(self):
        super(CombinedModel, self).__init__()
        
        self.cnn_branch = CNNBranch()
        self.ancillary_branch = AncillaryBranch()
        
        self.combined_regressor = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 6)
        )
    
    def forward(self, image, ancillary):
        image_features = self.cnn_branch(image)
        ancillary_features = self.ancillary_branch(ancillary)
        
        combined_features = torch.cat((image_features, ancillary_features), dim=1)
        
        x = self.combined_regressor(combined_features) 
        return x

## Training

In [13]:
model = CombinedModel()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.MSELoss()
model.to(device)
print("Model Created")

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 148MB/s]


Model Created


In [14]:
def train(model, device, train_loader, criterion, optimizer, epoch):
    loss_fn = criterion
    model.train()
    train_loss_total = 0
    for batch_idx, (data, label, target) in enumerate(train_loader):
        data, label, target = data.to(device), label.to(device),target.to(device)
        optimizer.zero_grad()
        output = model(data, label)
        loss = loss_fn(output, target)
        
        '''reg_loss = 0
        for param in model.parameters():
            reg_loss += torch.norm(param, 1)
        loss += 0.3 * reg_loss'''
        
        loss.backward()
        train_loss_total += loss.item()
        optimizer.step()
        
        if (batch_idx + 1) % 100 == 0:
            print('[{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    print("Total training loss: ", train_loss_total)
    print("Number of data points: ", len(train_loader.dataset))
    train_loss = train_loss_total / (len(train_loader.dataset))

    print('Training set Average loss: {}'.format(train_loss))
    return train_loss


def validate(model, device, validate_loader, criterion, epoch):
    loss_fn = criterion
    model.eval()
    validate_loss_total = 0
    with torch.no_grad():
        for batch_idx, (data, label, target) in enumerate(validate_loader):
            data, label, target = data.to(device), label.to(device),target.to(device)
            output = model(data, label)
            loss = loss_fn(output, target)
            validate_loss_total += loss.item()
        
        print("Total validation loss: ", validate_loss_total)
        print("Number of data points: ", len(validate_loader.dataset))
        validate_loss = validate_loss_total / len(validate_loader.dataset)
        print('Validation set Average loss: ', validate_loss)
        return validate_loss

In [15]:
num_epochs = 20
training_loss = [0 for _ in range(num_epochs)]
validation_loss = [0 for _ in range(num_epochs)]
best_val_loss = 1000000000
overfit_count = 3
for epoch in range(1, num_epochs + 1):
    print("Epoch: ", epoch)
    print("---------------------------------------------")

    train_loss = train(model, device, trainloader, criterion, optimizer, epoch)
    val_loss = validate(model, device, validationloader, criterion, epoch)
    
    training_loss[epoch - 1] = train_loss
    validation_loss[epoch - 1] = val_loss
    
    if val_loss < best_val_loss:
        print("Best so far is epoch ", epoch)
        best_val_loss = val_loss
        overfit_count = 2
        torch.save(model.state_dict(), 'best_weights.pth')
    else:
        overfit_count -= 1
    
    if overfit_count == 0:
        print("---------------------------------------------")
        print('Early stopping')
        print("---------------------------------------------")
        break
    
    if epoch % 3 == 0:
        torch.save(model.state_dict(), 'epoch{}.pth'.format(epoch))
        print(f'Model saved at epoch {epoch}')
        

    print("---------------------------------------------")

print('Finished Training')

Epoch:  1
---------------------------------------------
[3200/39027 (8%)]	Loss: 0.059625
[6400/39027 (16%)]	Loss: 0.049250
[9600/39027 (25%)]	Loss: 0.061322
[12800/39027 (33%)]	Loss: 0.039057
[16000/39027 (41%)]	Loss: 0.028155
[19200/39027 (49%)]	Loss: 0.039481
[22400/39027 (57%)]	Loss: 0.033516
[25600/39027 (65%)]	Loss: 0.026493
[28800/39027 (74%)]	Loss: 0.034966
[32000/39027 (82%)]	Loss: 0.037983
[35200/39027 (90%)]	Loss: 0.019289
[38400/39027 (98%)]	Loss: 0.029939
Total training loss:  55.87343001924455
Number of data points:  39027
Training set Average loss: 0.0014316609019203257
Total validation loss:  3.657795624807477
Number of data points:  4336
Validation set Average loss:  0.0008435875518467428
Best so far is epoch  1
---------------------------------------------
Epoch:  2
---------------------------------------------
[3200/39027 (8%)]	Loss: 0.033501
[6400/39027 (16%)]	Loss: 0.028735
[9600/39027 (25%)]	Loss: 0.026456
[12800/39027 (33%)]	Loss: 0.031024
[16000/39027 (41%)]	Loss

### Fine Tuning

# Evaluate

In [16]:
PATH = '/kaggle/working/best_weights.pth'
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [17]:
def r2score(model, device, dataloader):
    predictions = []
    targets = []
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, label, target) in enumerate(dataloader):
            data, label, target = data.to(device), label.to(device), target.to(device)
            output = model(data, label)
            predictions.extend(output.cpu().numpy())
            targets.extend(target.cpu().numpy())
        
        targets = np.array(targets)
        predictions = np.array(predictions)

        scores = []
        for i in range(6):
            r2 = r2_score(targets[:, i], predictions[:, i])
            scores.append(r2)
            
        print(scores)
        return np.mean(scores)
            

In [18]:
r2score(model, device, validationloader)

[0.21912794553760018, 0.3039815472441173, 0.4625236986787118, 0.20287494312721444, 0.18124637105693175, 0.30638828709622123]


0.2793571321234661

# Predict

In [19]:
class TestImageSet(Dataset):
    def __init__(self, root_dir, indices, label_mapping, target_mapping, transform = None):
        self.root_dir = root_dir
        self.indices = indices
        self.transform = transform
        self.image_paths = []
        self.targets = []
        self.labels = []
        
        for idx in indices:
            self.image_paths.append(os.path.join(root_dir, str(idx) + ".jpeg"))
            target = self.get_target_from_filename(idx, target_mapping)
            self.targets.append(target)
            label = self.get_label_from_filename(idx, label_mapping)
            self.labels.append(label)
                
    def get_target_from_filename(self, idx, target_mapping):
        return int(target_mapping.loc[idx].values[0])
    
    def get_label_from_filename(self, idx, label_mapping):
        target = label_mapping.loc[idx].values
        return torch.tensor(target).float()
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        target = self.targets[idx]
        label = self.labels[idx]
 
        if self.transform:
            image = self.transform(image)

        return image, label, target

In [20]:
test_data = pd.read_csv("/kaggle/input/cs-480-2024-spring/data/test.csv")
test_data = test_data.set_index('id')

test_data_normalized = (test_data - data_min) / (data_max - data_min)

# Mapping for IDs
test_data_index = pd.DataFrame(index = test_data.index)
test_data_index['id'] = test_data.index
test_data_index['id'] = test_data_index['id'].astype(int)

In [21]:
'''print(test_data_index['id'].nunique())
print(test_data.index.nunique())
print(type(test_data.index))'''

"print(test_data_index['id'].nunique())\nprint(test_data.index.nunique())\nprint(type(test_data.index))"

In [22]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


testset = TestImageSet('/kaggle/input/cs-480-2024-spring/data/test_images/', test_data.index.tolist(), label_mapping = test_data_normalized,
                               target_mapping = test_data_index, transform=test_transform)

batch_size = 1

testloader = DataLoader(testset, batch_size = batch_size, shuffle=False)

In [23]:
columns = ['id', 'X4', 'X11', 'X18', 'X26', 'X50', 'X3112']
def predict(model, device, test_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for data, label, target in test_loader:
            data, label, target = data.to(device), label.to(device), target.to(device)
            output = model(data, label)
            output = inverse_transformation(output.cpu().numpy()[0])
            target = int(target.cpu().numpy()[0])
            
            new_row = pd.Series([target] + output.tolist(), index=columns)
            predictions.append(new_row)
            
    return predictions

In [24]:
predictions = predict(model, device, testloader)
df = pd.DataFrame(predictions)

In [25]:
'''print(df['id'].nunique())
print(df.shape)

print(df['id'].duplicated())'''
# print(df.head())

"print(df['id'].nunique())\nprint(df.shape)\n\nprint(df['id'].duplicated())"

In [26]:
df.to_csv('submission.csv', index=False)